In [12]:
# Google Colab에서 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

import time
from music21 import converter, instrument, note, chord, stream, midi
import glob
import numpy as np
import pandas as pd

# Melody-RNN Format is a sequence of 8-bit integers indicating the following:
# MELODY_NOTE_ON = [0, 127] # (note on at that MIDI pitch)
MELODY_NOTE_OFF = 83 # (stop playing all previous notes)
MELODY_NO_EVENT = 84 # (no change from previous event)

# Each element in the sequence lasts for one sixteenth note.
# This can encode monophonic music only
def streamToNoteArray(stream):
   """
   Convert a Music21 sequence to a numpy array of int8s into Melody-RNN format:
   0-127 - note on at specified pitch
   128 - note off
   129 - no event
   """
   # Part one, extract from stream
   total_length = np.int(np.round(stream.flat.highestTime / 0.25)) # in semiquavers
   stream_list = []
   for element in stream.flat:
       if isinstance(element, note.Note):
           stream_list.append([np.round(element.offset / 0.25), np.round(element.quarterLength / 0.25), element.pitch.midi])
       elif isinstance(element, chord.Chord):
           stream_list.append([np.round(element.offset / 0.25), np.round(element.quarterLength / 0.25), element.sortAscending().pitches[-1].midi])
   np_stream_list = np.array(stream_list, dtype=np.int)
   df = pd.DataFrame({'pos': np_stream_list.T[0], 'dur': np_stream_list.T[1], 'pitch': np_stream_list.T[2]})
   df = df.sort_values(['pos','pitch'], ascending=[True, False]) # sort the dataframe properly
   df = df.drop_duplicates(subset=['pos']) # drop duplicate values

   # part 2, convert into a sequence of note events
   output = np.zeros(total_length+1, dtype=np.int16) + np.int16(MELODY_NO_EVENT) # set array full of no events by default.
   # Fill in the output list
   for i in range(total_length):
       if not df[df.pos==i].empty:
           n = df[df.pos==i].iloc[0] # pick the highest pitch at each semiquaver
           output[i] = n.pitch # set note on
           output[i+n.dur] = MELODY_NOTE_OFF
   return output

# 기존 코드에서 사용한 MIDI 파일 경로
midi_files = glob.glob("/content/drive/MyDrive/GAN/lofi/*.mid")
print(midi_files)


new_midi_files = glob.glob("/content/path/to/your/midi/files/*.mid")

training_arrays = []
for f in midi_files + new_midi_files:
   start = time.perf_counter()
   try:
       s = converter.parse(f)
   except:
       continue
   arr = streamToNoteArray(s.parts[0]) # just extract first voice
   arr = np.where(arr > 84, 84, arr)
   value = arr.shape[0] // 4
   arr = arr[:value * 4]
   arr = arr.reshape(-1, 4)
   arr = arr.astype(np.float16)
   training_arrays.append(arr)
   print("Converted:", f, "it took", time.perf_counter() - start)

training_dataset = np.array(training_arrays)
np.savez('/content/drive/MyDrive/GAN/lo-fi.npz', train=training_dataset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[]


In [13]:
# Google Colab에서 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

import time
from music21 import converter, instrument, note, chord, stream, midi
import glob
import numpy as np
import pandas as pd

# Melody-RNN Format is a sequence of 8-bit integers indicating the following:
# MELODY_NOTE_ON = [0, 127] # (note on at that MIDI pitch)
MELODY_NOTE_OFF = 83 # (stop playing all previous notes)
MELODY_NO_EVENT = 84 # (no change from previous event)

# Each element in the sequence lasts for one sixteenth note.
# This can encode monophonic music only
def streamToNoteArray(stream):
    """
    Convert a Music21 sequence to a numpy array of int8s into Melody-RNN format:
    0-127 - note on at specified pitch
    128 - note off
    129 - no event
    """
    # Part one, extract from stream
    total_length = np.int(np.round(stream.flat.highestTime / 0.25)) # in semiquavers
    stream_list = []
    for element in stream.flat:
        if isinstance(element, note.Note):
            stream_list.append([np.round(element.offset / 0.25), np.round(element.quarterLength / 0.25), element.pitch.midi])
        elif isinstance(element, chord.Chord):
            stream_list.append([np.round(element.offset / 0.25), np.round(element.quarterLength / 0.25), element.sortAscending().pitches[-1].midi])
    np_stream_list = np.array(stream_list, dtype=np.int)
    df = pd.DataFrame({'pos': np_stream_list.T[0], 'dur': np_stream_list.T[1], 'pitch': np_stream_list.T[2]})
    df = df.sort_values(['pos','pitch'], ascending=[True, False]) # sort the dataframe properly
    df = df.drop_duplicates(subset=['pos']) # drop duplicate values

    # part 2, convert into a sequence of note events
    output = np.zeros(total_length+1, dtype=np.int16) + np.int16(MELODY_NO_EVENT) # set array full of no events by default.
    # Fill in the output list
    for i in range(total_length):
        if not df[df.pos==i].empty:
            n = df[df.pos==i].iloc[0] # pick the highest pitch at each semiquaver
            output[i] = n.pitch # set note on
            output[i+n.dur] = MELODY_NOTE_OFF
    return output

# MIDI 파일에서 chords 추출
def extract_chords(midi_file):
    # MIDI 파일에서 코드 정보 추출 코드
    # 예시: 모든 동시에 연주되는 노트 조합을 코드로 간주
    midi_stream = converter.parse(midi_file)
    chords = []
    for element in midi_stream.flat:
        if isinstance(element, chord.Chord):
            chord_pitches = [p.midi for p in element.pitches]
            chords.append(chord_pitches)
    return chords

# MIDI 파일에서 style 추출 (가정: 템포와 박자를 스타일로 사용)
def extract_style(midi_file):
    midi_stream = converter.parse(midi_file)
    tempo = midi_stream.metronomeMarkBoundaries()[0][2].getNumber()
    time_signature = midi_stream.flattenedParts()[0].getTimeSignatures()[0]
    style = [tempo, time_signature.numerator, time_signature.denominator]
    return style

# MIDI 파일에서 melody 추출
def extract_melody(midi_file):
    midi_stream = converter.parse(midi_file)
    melody_stream = midi_stream.parts[0]  # 가정: 첫 번째 파트가 멜로디
    melody = streamToNoteArray(melody_stream)
    return melody

# MIDI 파일에서 groove 추출 (가정: 박자 패턴을 groove로 사용)
def extract_groove(midi_file):
    midi_stream = converter.parse(midi_file)
    groove = []
    for element in midi_stream.flat:
        if isinstance(element, note.Note):
            groove.append(element.offset)
    return groove

# MIDI 파일 경로
midi_files = glob.glob("/content/drive/MyDrive/GAN/lofi/*.mid")

# museGAN 입력값 전처리 함수 (가정)
def preprocess_chords(chords):
    # 코드 전처리 코드
    return preprocessed_chords

def preprocess_style(style):
    # 스타일 전처리 코드
    return preprocessed_style

def preprocess_melody(melody):
    # 멜로디 전처리 코드
    return preprocessed_melody

def preprocess_groove(groove):
    # 그루브 전처리 코드
    return preprocessed_groove

# museGAN 모델 (가정)
def museGAN_model(chords, style, melody, groove):
    # museGAN 모델 코드
    return output

for f in midi_files:
    chords = extract_chords(f)
    style = extract_style(f)
    melody = extract_melody(f)
    groove = extract_groove(f)

    chords = preprocess_chords(chords)
    style = preprocess_style(style)
    melody = preprocess_melody(melody)
    groove = preprocess_groove(groove)

    output = museGAN_model(chords, style, melody, groove)
    print("Generated output for", f)
    # 출력 결과 처리 코드

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
